# Capstone: Recommending Art
By: Corina Lentz

For my capstone I will be building a recommender engine that can take an input of an artist and (using associated art tags) recommend other artists the user might also like. Ideally I would like to create a live demo of the recommender with Streamlit and host it on Heroku (time allowing).

I'm confident that this recommender would be valuable for museums to drive engagement on their social media and websites, which could then be used to increase museum memberships, special exhibit ticket sales, and online shop sales for related merchandise.

The dataset I will be using for this project is the Metropolitan Museum of Art Open Access dataset and it has over 475,000 art pieces included in it. It can be found on GitHub [here](https://github.com/metmuseum/openaccess).  

Initial EDA suggests only a small portion of the data will be useful for this project, however since the dataset is so large to start with I should still have plent to work with once data cleaning and pre-processing is done. Another concern is that there are no images of the actual artwork included with this dataset, but rather links to webpages where the art can be viewed.

Alternative data: web scraping https://www.wikiart.org/  or https://github.com/cs-chan/ArtGAN/tree/master/WikiArt%20Dataset 

## Exploring the Dataset

In [2]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [27]:
#Referenced this page for code to adjust the max columns: 
#https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf
#And referenced this page for code to adjust max column width:
#https://stackoverflow.com/questions/29902714/print-very-long-string-completely-in-pandas-dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('./data/MetObjects.csv', low_memory = False)

In [5]:
df.head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN


In [6]:
df.shape

(475897, 54)

In [7]:
df.isnull().sum()

Object Number                   0
Is Highlight                    0
Is Timeline Work                0
Is Public Domain                0
Object ID                       0
Gallery Number             431969
Department                      0
AccessionYear                4306
Object Name                  1515
Title                       29306
Culture                    268875
Period                     385162
Dynasty                    452657
Reign                      464672
Portfolio                  452557
Constituent ID             202187
Artist Role                204286
Artist Prefix              202187
Artist Display Name        202187
Artist Display Bio         204286
Artist Suffix              202235
Artist Alpha Sort          202187
Artist Nationality         202187
Artist Begin Date          202187
Artist End Date            202187
Artist Gender              373955
Artist ULAN URL            257119
Artist Wikidata URL        260360
Object Date                 14333
Object Begin D

In [8]:
#Dropping all rows that have a null value in the Tags column and saving to a new df
tags=df.dropna(subset=['Tags'])
tags.head(5)

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
6,67.265.13,False,False,False,7,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/7,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Birds|Coins,http://vocab.getty.edu/page/aat/300266506|http...,https://www.wikidata.org/wiki/Q5113|https://ww...
7,67.265.14,False,False,False,8,NaN,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/8,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Eagles|Men|Profiles,http://vocab.getty.edu/page/aat/300250049|http...,https://www.wikidata.org/wiki/Q2092297|https:/...
32,64.62,False,False,False,33,774,The American Wing,1964,Bust,Bust of Abraham Lincoln,American,NaN,NaN,NaN,NaN,16459,Maker,,James Gillinder and Sons,"American, 1861–ca. 1930",,"Gillinder and Sons, James",American,1861,1930,NaN,NaN,NaN,1876,1876,1876,Pressed glass,2 3/4 x 3 1/2 x 2 3/4 in. (7 x 8.9 x 7 cm),"Gift of Teunis G. B. Cortelyou, 1964",Made in,Philadelphia,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/33,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Men|Abraham Lincoln|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
33,1970.289.6,False,False,True,34,774,The American Wing,1970,Clock,Acorn Clock,American,NaN,NaN,NaN,NaN,108,Maker,,Forestville Manufacturing Company,1835–1853,,Forestville Manufacturing Company,,1835,1853,NaN,NaN,NaN,1847–50,1847,1850,"Mahogany, laminated",24 3/8 x 14 5/8 x 5 1/8 in. (61.9 x 37.1 x 13 cm),"Gift of Mrs. Paul Moore, 1970",Made in,Bristol,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/34,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Landscapes|Boats,http://vocab.getty.edu/page/aat/300132294|http...,https://www.wikidata.org/wiki/Q191163|https://...
34,04.1a–c,True,True,False,35,706,The American Wing,1904,Vase,The Adams Vase,American,NaN,NaN,NaN,NaN,108316253,Designer|Manufacturer,Designed by|Manufactured by,Paulding Farnham|Tiffany & Co.,1859–1927|1837–present,|,"Farnham, Paulding|Tiffany & Co.",|,1859 |1837,1927 |9999,|,http://vocab.getty.edu/page/ulan/500336597|htt...,https://www.wikidata.org/wiki/Q13476260|https:...,1893–95,1893,1895,"Gold, amethysts, spessartites, tourmalines, fr...",Overall: 19 7/16 x 13 x 9 1/4 in. (49.4 x 33 x...,"Gift of Edward D. Adams, 1904",Made in,New York,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/35,https://www.wikidata.org/wiki/Q83545838,NaN,"Metropolitan Museum of Art, New York, NY",Animals|Garlands|Birds|Men,http://vocab.getty.edu/page/aat/300249525|http...,https://www.wikidata.org/wiki/Q729|https://www...


In [9]:
#Also dropping all rows with null values in 'Artist Display Name'
tags=df.dropna(subset=['Artist Display Name'])

In [10]:
#Saving the new dataframe
tags.to_csv('../data-with-tags.csv')

In [9]:
tags.shape

(273710, 54)

Even after dropping all those rows there is still a large amount of data to work with.

In [11]:
#Taking a look at the value counts in the Department column.
tags['Department'].value_counts()

Drawings and Prints                          160552
Photographs                                   36171
Costume Institute                             16428
Modern and Contemporary Art                   14064
European Sculpture and Decorative Arts        13783
The American Wing                             11879
Asian Art                                      9610
European Paintings                             2615
Greek and Roman Art                            2157
Robert Lehman Collection                       1465
Musical Instruments                            1247
Arms and Armor                                  990
Arts of Africa, Oceania, and the Americas       875
Islamic Art                                     708
Egyptian Art                                    548
The Libraries                                   325
Medieval Art                                    162
The Cloisters                                   125
Ancient Near Eastern Art                          6
Name: Depart

In [12]:
#Taking a look at the artists
tags['Artist Display Name'].value_counts(ascending=True)

Hurst, Robinson & Co.|Robert Jennings, London|Robert Smirke|Charles Heath, the elder|William Shakespeare       1
Frederick Landseer Griggs                                                                                      1
Seireniske Painter                                                                                             1
Johann Wilhem Heel                                                                                             1
William Makepeace Thackeray|Bradley & Evans                                                                    1
                                                                                                            ... 
Allen & Ginter                                                                                              2960
W. Duke, Sons & Co.                                                                                         3088
Unknown                                                                                         

Using .loc to drop any rows that are from the 'Egyptian Art', 'Costume Institute', 'Arms and Armor', 'Musical Instruments', and 'Photographs' departments. Ancient Egypt, clothing design, armor/weapons, musical instruments, and photographs are outside the scope of this project.

In [12]:
#Referenced this site for filtering code: https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
tags=tags.loc[tags['Department'] != 'Egyptian Art']

tags=tags.loc[tags['Department'] != 'Costume Institute']

tags=tags.loc[tags['Department'] != 'Arms and Armor']

tags=tags.loc[tags['Department'] != 'Musical Instruments']

tags=tags.loc[tags['Department'] != 'Photographs']

In [13]:
#Confirming it filtered it out
tags['Department'].value_counts()

Drawings and Prints                          160552
Modern and Contemporary Art                   14064
European Sculpture and Decorative Arts        13783
The American Wing                             11879
Asian Art                                      9610
European Paintings                             2615
Greek and Roman Art                            2157
Robert Lehman Collection                       1465
Arts of Africa, Oceania, and the Americas       875
Islamic Art                                     708
The Libraries                                   325
Medieval Art                                    162
The Cloisters                                   125
Ancient Near Eastern Art                          6
Name: Department, dtype: int64

Now I'll take a look at specific departments to see if the art can be used for this project.

In [29]:
#Starting with Ancient Near Eastern Art 
tags[tags['Department']== 'Ancient Near Eastern Art']

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
201669,1998.6.1,False,False,True,327533,NaN,Ancient Near Eastern Art,1998,Drawing,"Drawing of Sasanian rock relief: Shapur I (r. A.D. 241-272) and members of his court at Naqsh-i Radjab, southern Iran",Qajar,NaN,NaN,NaN,NaN,6828,Artist,,Lutf 'Ali Khan,1797–1869,,Lutf 'Ali Khan of Shiraz,,1797,1869,NaN,NaN,NaN,A.D. 1860 (drawing),1860,1860,"Paper, pencil, ink",13.78 x 15.75 in. (35 x 40.01 cm),"Gift of Mr. Sassan Mokhtarzadeh, 1998",NaN,NaN,NaN,NaN,NaN,Iran,NaN,NaN,NaN,NaN,NaN,Paper-Graphics-Inscribed,NaN,http://www.metmuseum.org/art/collection/search/327533,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Horse Riding|Men,http://vocab.getty.edu/page/aat/300249313|http://vocab.getty.edu/page/aat/300025928,https://www.wikidata.org/wiki/Q179226|https://www.wikidata.org/wiki/Q8441
201670,1998.6.2,False,False,True,327534,NaN,Ancient Near Eastern Art,1998,Drawing,Drawing of Sasanian rock relief,Qajar,NaN,NaN,NaN,NaN,6828,Artist,,Lutf 'Ali Khan,1797–1869,,Lutf 'Ali Khan of Shiraz,,1797,1869,NaN,NaN,NaN,A.D. 1860 (drawing),1860,1860,"Paper, ink",9.72 x 13.78 in. (24.69 x 35 cm),"Gift of Mr. Sassan Mokhtarzadeh, 1998",NaN,NaN,NaN,NaN,NaN,Iran,NaN,NaN,NaN,NaN,NaN,Paper-Graphics,NaN,http://www.metmuseum.org/art/collection/search/327534,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Soldiers|Horse Riding,http://vocab.getty.edu/page/aat/300185678|http://vocab.getty.edu/page/aat/300249313,https://www.wikidata.org/wiki/Q4991371|https://www.wikidata.org/wiki/Q179226
201671,1998.6.3,False,True,True,327535,405,Ancient Near Eastern Art,1998,Drawing,"Drawing of Sasanian rock relief: Ardashir I (r. A.D. 224-241) and the Zoroastrian divinity Ahura Mazda at Naqsh-i Rustam, southern Iran",Qajar,NaN,NaN,NaN,NaN,6828,Artist,,Lutf 'Ali Khan,1797–1869,,Lutf 'Ali Khan of Shiraz,,1797,1869,NaN,NaN,NaN,A.D. 1860 (drawing),1860,1860,"Paper, pencil, ink",12.76 x 15.75 in. (32.41 x 40.01 cm),"Gift of Mr. Sassan Mokhtarzadeh, 1998",NaN,NaN,NaN,NaN,NaN,Iran,NaN,NaN,NaN,NaN,NaN,Paper-Graphics,NaN,http://www.metmuseum.org/art/collection/search/327535,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Kings|Men|Horse Riding,http://vocab.getty.edu/page/aat/300025481|http://vocab.getty.edu/page/aat/300025928|http://vocab.getty.edu/page/aat/300249313,https://www.wikidata.org/wiki/Q12097|https://www.wikidata.org/wiki/Q8441|https://www.wikidata.org/wiki/Q179226
201672,1998.6.4,False,False,True,327536,NaN,Ancient Near Eastern Art,1998,Drawing,Drawing of Sasanian rock relief,Qajar,NaN,NaN,NaN,NaN,6828,Artist,,Lutf 'Ali Khan,1797–1869,,Lutf 'Ali Khan of Shiraz,,1797,1869,NaN,NaN,NaN,A.D. 1860 (drawing),1860,1860,"Paper, ink",12.76 x 15.75 in. (32.41 x 40.01 cm),"Gift of Mr. Sassan Mokhtarzadeh, 1998",NaN,NaN,NaN,NaN,NaN,Iran,NaN,NaN,NaN,NaN,NaN,Paper-Graphics,NaN,http://www.metmuseum.org/art/collection/search/327536,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Swords|Men|Horse Riding,http://vocab.getty.edu/page/aat/300037048|http://vocab.getty.edu/page/aat/300025928|http://vocab.getty.edu/page/aat/300249313,https://www.wikidata.org/wiki/Q12791|https://www.wikidata.org/wiki/Q8441|https://www.wikidata.org/wiki/Q179226
456578,2019.116,False,False,True,781871,NaN,Ancient Near Eastern Art,2019,Persian Travelogue,Persian Travelogue: A Diary of a Journey through the region of Fars,Qajar,NaN,NaN,NaN,

This section contains art from three artists. A brief look at the provided URL for the first piece of art (http://www.metmuseum.org/art/collection/search/327533) and a review of the art descriptions confirmed that these art pieces are within the scope of this project. 

In [31]:
#Reviewing The Libraries department and starting with the first 5 rows.
tags[tags['Department']== 'The Libraries'].head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
366265,125.97 D932,True,False,False,591824,NaN,The Libraries,NaN,NaN,"""Alberti Dvreri pictoris et architecti praestantissimi De vrbibvs...""","Paris: Officina Christiani Wecheli, 1535",NaN,NaN,NaN,NaN,16185,Author,,Albrecht Dürer,"German, Nuremberg 1471–1528 Nuremberg",,"Dürer, Albrecht",German,1471,1528,NaN,http://vocab.getty.edu/page/ulan/500115493,https://www.wikidata.org/wiki/Q5580,1535,1535,1535,Illustrated book,78 pp.; H: 13 3/4 in. (35 cm),Purchased with income from the Jacob S. Rogers Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/591824,https://www.wikidata.org/wiki/Q29385974,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
366266,131.1M58 C75,True,False,False,591825,NaN,The Libraries,NaN,NaN,Vita di Michelagnolo Buonarroti raccolta per Ascanio Condivi da la Ripa Transone,"Rome: Antonio Blado, 1553 (1st ed.)",NaN,NaN,NaN,NaN,16141,Author,,Ascanio Condivi,"Italian, Ripatransone 1525–1574 Ripatransone",,"Condivi, Ascanio",Italian,1525,1574,NaN,http://vocab.getty.edu/page/ulan/500115521,https://www.wikidata.org/wiki/Q721545,1553,1553,1553,Illustrated book,H: 7 7/8 in. (20 cm),Purchased with income from the Jacob S. Rogers Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/591825,https://www.wikidata.org/wiki/Q29385977,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
366267,175T49 R43,True,False,False,591826,NaN,The Libraries,NaN,NaN,Vita del Tintoretto,"Venice: Oddoni, 1642",NaN,NaN,NaN,NaN,16141,Author,,Carlo Ridolfi,"Italian, Lonigo 1594–1658 Venice",,"Ridolfi, Carlo",Italian,1594,1658,NaN,http://vocab.getty.edu/page/ulan/500013305,https://www.wikidata.org/wiki/Q776990,1642,1642,1642,Illustrated book,H: 9 in. (23 cm),Purchased with income from the Jacob S. Rogers Fund,|,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/591826,https://www.wikidata.org/wiki/Q29385980,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
366268,120 Se6,True,False,False,591827,NaN,The Libraries,NaN,NaN,"Bononiensis de architectura libri quinque quibus cuncta fere architectonicae facultatis mysteria docte, perspicue . . .",NaN,NaN,NaN,NaN,NaN,16141,Author,,Sebastiano Serlio,"Italian, Bologna 1475–1554 Fontainebleau",,"Serlio, Sebastiano",Italian,1475,1554,NaN,http://vocab.getty.edu/page/ulan/500023485,https://www.wikidata.org/wiki/Q452318,1568–69,1568,1569,Illustrated book,5 pts. in 1 vol.; H: 12 5/8 in. (32 cm),Purchased with income from the Jacob S. Rogers Fund,NaN,Venice: Apud Francifcum de Francifcis Senenfem,NaN,"& Joanneum Chriegher, 1568–1569",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/591827,https://www.wikidata.org/wiki/Q29385983,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
366269,120.32P17 P17,True,False,False,591828,NaN,The Libraries,NaN,NaN,I quattro libri dell'architettura di Andrea Palladio . . .,"Venice: Domenico de'Franceschi, 1570",NaN,NaN,NaN,NaN,16142,Author,,Andrea Palladio,"Italian, Padua 1508–1580 Vicenza",,"Palladio, Andrea",Italian,1508,1580,NaN,http://vocab.getty.edu/page/ulan/500021650,https://www.wikidata.org/wiki/Q177692,1570,1570,1570,Illustrated book,"4 pts. in 1 vol., 128 pp.; H: 4 3/4 in. (12 cm)",Library Purcha

It looks like The Libraries department may only contain illustrated books, which is outside the scope of this project. I'll take a look at the unique Medium column values to see if that is the case. 

In [34]:
tags.groupby(by=[tags['Department']=='The Libraries'])['Medium'].unique()

Department
False    [Gold, Pressed glass, Mahogany, laminated, Gold, amethysts, spessartites, tourmalines, fresh water pearls, quartzes, rock crystal, and enamel, Oak, tulip poplar, Earthenware, basalt, Brass and iron, Brass, Brass, iron, Bell metal, Bronze, glass, iron, Wood, gilt, Wood, Limestone, ceramic, and Fravrile glass, Bronze, gilt brass, Gilt bronze, Graphite on off-white wove paper, Graphite and pen and ink on off-white wove paper, Soft maple, poplar, ash, Silver, Walnut, Mahogany, Mahogany, white oak, Mahogany, ash, Oak, poplar, Maple, Rosewood, marquetry, gilt, Rattan, maple, possibly ash, Maple, pine, mahogany, Cherry, Maple, steel mesh, Oak, leather, Maple, gilt bronze, Walnut, metal, Mahogany, birch, white pine, Cast iron, Rosewood, Black walnut, Black walnut, original and reproduction upholstery, Rosewood, prickly juniper veneer, Oak, Prima vera, American ash (secondary wood); marquetry of various woods and brass; replacement upholstery, Steel, wood, Rosewood, ash, mot

The Libraries department includes illustrated books, printed books, engraved plates, letters, lithographs, bound manuscripts, and other literary items which are all outside the scope of this project. Dropping this department from our dataset. 

In [36]:
tags=tags.loc[tags['Department'] != 'The Libraries']

In [37]:
#Reviewing The Cloisters department
tags[tags['Department'] == 'The Cloisters'].head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
313075,09.146,False,False,True,463192,016,The Cloisters,1909,Altarpiece,Altar Predella and Socle of Archbishop Don Dalmau de Mur y Cervelló,Spanish,NaN,NaN,NaN,NaN,13886,Artist,,Francí Gomar,"Spanish, Aragon, active by 1443–died ca. 1492/3",,Gomar Francí,Spanish,1443,1493,NaN,NaN,https://www.wikidata.org/wiki/Q56715821,ca. 1456–1458,1453,1461,Alabaster with traces of paint and gilding,107 x 183 x 26 1/4 in. (271.8 x 464.8 x 66.7 cm),"Gift of J. Pierpont Morgan, 1909",Made in,Saragossa,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture-Stone,NaN,http://www.metmuseum.org/art/collection/search/463192,https://www.wikidata.org/wiki/Q20200912,NaN,"Metropolitan Museum of Art, New York, NY",Virgin Mary|Apostles|Saints,http://vocab.getty.edu/page/ia/901000032|http://vocab.getty.edu/page/aat/300343837|http://vocab.getty.edu/page/aat/300150555,https://www.wikidata.org/wiki/Q345|https://www.wikidata.org/wiki/Q43412|https://www.wikidata.org/wiki/Q43115
313227,"10.12a, b",False,False,True,463377,020,The Cloisters,1910,Altarpiece,Altarpiece (retablo) with Scenes from the Passion,Spanish,NaN,NaN,NaN,NaN,14245,Artist,Attributed to,Master Morata,"Spanish, Aragon, active 1470–1505",,Master Morata,Spanish,1470,1505,NaN,http://vocab.getty.edu/page/ulan/500039615,https://www.wikidata.org/wiki/Q101530066,second half 15th century,1450,1499,"Tempera, gilt, wood","Overall: 56 x 120 3/4 x 3 in., 178lb. (142.2 x 306.7 x 7.6 cm, 80740.3g)","Gift of W. M. Laffan, 1910",Made in,NaN,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Paintings-Panels,NaN,http://www.metmuseum.org/art/collection/search/463377,https://www.wikidata.org/wiki/Q20200947,NaN,"Metropolitan Museum of Art, New York, NY",Men|Women|Judas|Christ,http://vocab.getty.edu/page/aat/300025928|http://vocab.getty.edu/page/aat/300025943|http://vocab.getty.edu/page/ia/901000531|http://vocab.getty.edu/page/ia/901000087,https://www.wikidata.org/wiki/Q8441|https://www.wikidata.org/wiki/Q467|https://www.wikidata.org/wiki/Q81018|https://www.wikidata.org/wiki/Q642420
313537,16.79,False,False,True,463958,016,The Cloisters,1916,Altarpiece,Altar Predella and Socle of Archbishop Don Dalmau de Mur y Cervelló,Spanish,NaN,NaN,NaN,NaN,13886,Artist,,Francí Gomar,"Spanish, Aragon, active by 1443–died ca. 1492/3",,Gomar Francí,Spanish,1443,1493,NaN,NaN,https://www.wikidata.org/wiki/Q56715821,ca. 1456–1458,1453,1461,Alabaster,7 3/4 × 7 1/2 × 6 1/4 in. (19.7 × 19.1 × 15.9 cm),"Gift of Emile Pares, 1916",Made in,Saragossa,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture-Stone,NaN,http://www.metmuseum.org/art/collection/search/463958,https://www.wikidata.org/wiki/Q20200929,NaN,"Metropolitan Museum of Art, New York, NY",Saint Paul,http://vocab.getty.edu/page/ia/901001058,https://www.wikidata.org/wiki/Q9200
315470,1991.232.15,False,True,True,466196,NaN,The Cloisters,1991,Manuscript leaf,Leaf from the Epistle to the Hebrews,Byzantine,NaN,NaN,NaN,NaN,13894,Artist,,Joannes Koulix,,",scribe","Koulix, Joannes",,,,NaN,NaN,NaN,1101,1101,1101,"Tempera, gold and ink on parchment",7 31/32 x 6 1/4 in. (20.3 x 16.0 cm),"Purchase, The Cloisters Collection, Rogers and Harris Brisbane Dick Funds, and Joseph Pulitzer Bequest, 1991",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manuscripts and Illuminations,NaN,http://www.metmuseum.org/art/collection/search/466196,https://www.wik

The Cloisters include sculptures and paintings, which are part of this project, however there are also illuminated manuscripts which are outside the scope of this project. The Classification column maybe useful in filtering-out these out.

In [40]:
#Taking a look at the unique values for the Classification column
tags.groupby(by = [tags['Department']=='The Cloisters'])['Classification'].unique()

Department
False    [nan, |, Ceramics, Daggers, Swords, Firearms-Pistols, Firearms-Pistols-Flintlock, Armor for Horse and Man, Firearms-Pistols-Wheellock, Archery Equipment-Crossbows, Shafted Weapons, Knives, Combination Weapons, Firearms-Guns-Wheellock, Firearms-Guns, Firearms Accessories-Flasks & Primers, Helmets, Armor for Man-1/2 Armor, Armor for Man, Armor Parts-Masks, Firearms-Hand Cannon, Archery Equipment-Bows, Miscellaneous-Ironwork, Miscellaneous, Armor Parts-Arms & Shoulders, Armor Parts-Breastplates, Equestrian Equipment, Armor Parts-Gauntlets, Firearms, Firearms Accessories, Equestrian Equipment-Shaffrons, Firearms-Guns-Matchlock, Equestrian Equipment-Stirrups, Armor-Miniatures, Firearms-Guns-Miquelet, Sword Blades, Armor Parts-Sollerets, Armor Parts, Armor Parts-Knee Defenses, Swords-Hunting, Armor Parts-Buffes, Sword Furniture-Tsuba, Sword Furniture-Kozuka, Helmets Parts, Armor Parts-Backplates, Armor for Man-3/4 Armor, Armor Parts-Thigh and Leg Defense, Firearms Accesso

In [55]:
#Creating a list that can be used to filter the unwanted classifications
clss_list = ['Manuscripts and Illuminations', 'Glass-Stained', 'Metalwork-Silver', 'Textiles-Tapestries', 'Textiles-Embroidered', 'Glass-Painted',
        'Metalwork-Copper alloy', 'Ivories-Elephant', 'Metalwork-Copper', 'Enamels-Basse taille', 'Metalwork-Brass', 'Installations', 'Enamels',
        'Glass-Miscellany', 'Enamels-Champlevé & Cloisonné']

filtr = '|'.join(clss_list) 

In [64]:
#Dropping any rows with any of the classification in filtr using code found here: 
#https://stackoverflow.com/questions/17097643/search-for-does-not-contain-on-a-dataframe-in-pandas
tags.loc[~tags['Classification'].str.contains(filtr)]

TypeError: bad operand type for unary ~: 'float'

In [67]:
tags['Classification'].value_counts(dropna=False)

Prints                                     78972
NaN                                        28847
Photographs                                26762
Prints|Ephemera                            23944
Drawings                                   22550
                                           ...  
Ephemera|Letters                               1
Cut Paper|Ephemera|Prints|Drawings             1
Drawings|Ornament & Architecture|Prints        1
Ephemera|Books                                 1
Textiles-Rugs|Textiles-Embroidered             1
Name: Classification, Length: 724, dtype: int64

In [60]:
tags.groupby(by = [tags['Department']=='The Cloisters'])['Classification'].unique()

Department
False    [nan, |, Ceramics, Daggers, Swords, Firearms-Pistols, Firearms-Pistols-Flintlock, Armor for Horse and Man, Firearms-Pistols-Wheellock, Archery Equipment-Crossbows, Shafted Weapons, Knives, Combination Weapons, Firearms-Guns-Wheellock, Firearms-Guns, Firearms Accessories-Flasks & Primers, Helmets, Armor for Man-1/2 Armor, Armor for Man, Armor Parts-Masks, Firearms-Hand Cannon, Archery Equipment-Bows, Miscellaneous-Ironwork, Miscellaneous, Armor Parts-Arms & Shoulders, Armor Parts-Breastplates, Equestrian Equipment, Armor Parts-Gauntlets, Firearms, Firearms Accessories, Equestrian Equipment-Shaffrons, Firearms-Guns-Matchlock, Equestrian Equipment-Stirrups, Armor-Miniatures, Firearms-Guns-Miquelet, Sword Blades, Armor Parts-Sollerets, Armor Parts, Armor Parts-Knee Defenses, Swords-Hunting, Armor Parts-Buffes, Sword Furniture-Tsuba, Sword Furniture-Kozuka, Helmets Parts, Armor Parts-Backplates, Armor for Man-3/4 Armor, Armor Parts-Thigh and Leg Defense, Firearms Accesso

In [17]:
#Taking a look at the number of unique values in the Object Name column to see if we can filter with them
tags['Object Name'].nunique()

5900

In [63]:
tags[tags['Department']=='The Cloisters'].head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
313075,09.146,False,False,True,463192,016,The Cloisters,1909,Altarpiece,Altar Predella and Socle of Archbishop Don Dalmau de Mur y Cervelló,Spanish,NaN,NaN,NaN,NaN,13886,Artist,,Francí Gomar,"Spanish, Aragon, active by 1443–died ca. 1492/3",,Gomar Francí,Spanish,1443,1493,NaN,NaN,https://www.wikidata.org/wiki/Q56715821,ca. 1456–1458,1453,1461,Alabaster with traces of paint and gilding,107 x 183 x 26 1/4 in. (271.8 x 464.8 x 66.7 cm),"Gift of J. Pierpont Morgan, 1909",Made in,Saragossa,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture-Stone,NaN,http://www.metmuseum.org/art/collection/search/463192,https://www.wikidata.org/wiki/Q20200912,NaN,"Metropolitan Museum of Art, New York, NY",Virgin Mary|Apostles|Saints,http://vocab.getty.edu/page/ia/901000032|http://vocab.getty.edu/page/aat/300343837|http://vocab.getty.edu/page/aat/300150555,https://www.wikidata.org/wiki/Q345|https://www.wikidata.org/wiki/Q43412|https://www.wikidata.org/wiki/Q43115
313227,"10.12a, b",False,False,True,463377,020,The Cloisters,1910,Altarpiece,Altarpiece (retablo) with Scenes from the Passion,Spanish,NaN,NaN,NaN,NaN,14245,Artist,Attributed to,Master Morata,"Spanish, Aragon, active 1470–1505",,Master Morata,Spanish,1470,1505,NaN,http://vocab.getty.edu/page/ulan/500039615,https://www.wikidata.org/wiki/Q101530066,second half 15th century,1450,1499,"Tempera, gilt, wood","Overall: 56 x 120 3/4 x 3 in., 178lb. (142.2 x 306.7 x 7.6 cm, 80740.3g)","Gift of W. M. Laffan, 1910",Made in,NaN,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Paintings-Panels,NaN,http://www.metmuseum.org/art/collection/search/463377,https://www.wikidata.org/wiki/Q20200947,NaN,"Metropolitan Museum of Art, New York, NY",Men|Women|Judas|Christ,http://vocab.getty.edu/page/aat/300025928|http://vocab.getty.edu/page/aat/300025943|http://vocab.getty.edu/page/ia/901000531|http://vocab.getty.edu/page/ia/901000087,https://www.wikidata.org/wiki/Q8441|https://www.wikidata.org/wiki/Q467|https://www.wikidata.org/wiki/Q81018|https://www.wikidata.org/wiki/Q642420
313537,16.79,False,False,True,463958,016,The Cloisters,1916,Altarpiece,Altar Predella and Socle of Archbishop Don Dalmau de Mur y Cervelló,Spanish,NaN,NaN,NaN,NaN,13886,Artist,,Francí Gomar,"Spanish, Aragon, active by 1443–died ca. 1492/3",,Gomar Francí,Spanish,1443,1493,NaN,NaN,https://www.wikidata.org/wiki/Q56715821,ca. 1456–1458,1453,1461,Alabaster,7 3/4 × 7 1/2 × 6 1/4 in. (19.7 × 19.1 × 15.9 cm),"Gift of Emile Pares, 1916",Made in,Saragossa,NaN,Aragon,Spain,NaN,NaN,NaN,NaN,NaN,NaN,Sculpture-Stone,NaN,http://www.metmuseum.org/art/collection/search/463958,https://www.wikidata.org/wiki/Q20200929,NaN,"Metropolitan Museum of Art, New York, NY",Saint Paul,http://vocab.getty.edu/page/ia/901001058,https://www.wikidata.org/wiki/Q9200
315470,1991.232.15,False,True,True,466196,NaN,The Cloisters,1991,Manuscript leaf,Leaf from the Epistle to the Hebrews,Byzantine,NaN,NaN,NaN,NaN,13894,Artist,,Joannes Koulix,,",scribe","Koulix, Joannes",,,,NaN,NaN,NaN,1101,1101,1101,"Tempera, gold and ink on parchment",7 31/32 x 6 1/4 in. (20.3 x 16.0 cm),"Purchase, The Cloisters Collection, Rogers and Harris Brisbane Dick Funds, and Joseph Pulitzer Bequest, 1991",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manuscripts and Illuminations,NaN,http://www.metmuseum.org/art/collection/search/466196,https://www.wik